# 🛰️ TelloBoost Color-Based Object Tracking
Use color-based object tracking to make a DJI Tello Boost drone follow a colored object in real-time using OpenCV.

if the ipynb file below doesn't run, please run the object_following.py file as there could be some mistakes in this ipynb file

In [ ]:
from djitellopy import Tello
import cv2
import numpy as np

In [ ]:
# Define tracking and control parameters
width = 640
height = 480
deadZone = 100

# PID control for forward/back motion
TARGET_AREA = 7000
KP_FB = 0.0005
MAX_FB_SPEED = 50

startCounter = 0
dir = 0
frame_count = 0
detected_area = 0

In [ ]:
# Initialize Tello
me = Tello()
me.connect()

# Zero all velocities
me.for_back_velocity = 0
me.left_right_velocity = 0
me.up_down_velocity = 0
me.yaw_velocity = 0
me.speed = 0

print(f"Battery: {me.get_battery()}%")

me.streamoff()
me.streamon()

In [ ]:
def empty(a): pass

cv2.namedWindow("HSV")
cv2.resizeWindow("HSV",640,240)
cv2.createTrackbar("HUE Min","HSV",5,179,empty)
cv2.createTrackbar("HUE Max","HSV",54,179,empty)
cv2.createTrackbar("SAT Min","HSV",162,255,empty)
cv2.createTrackbar("SAT Max","HSV",255,255,empty)
cv2.createTrackbar("VALUE Min","HSV",90,255,empty)
cv2.createTrackbar("VALUE Max","HSV",255,255,empty)

cv2.namedWindow("Parameters")
cv2.resizeWindow("Parameters",640,240)
cv2.createTrackbar("Threshold1","Parameters",0,255,empty)
cv2.createTrackbar("Threshold2","Parameters",0,255,empty)
cv2.createTrackbar("Area","Parameters",1000,30000,empty)

In [ ]:
def stackImages(scale, imgArray):
    rows = len(imgArray)
    cols = len(imgArray[0])
    rowsAvailable = isinstance(imgArray[0], list)
    width = imgArray[0][0].shape[1]
    height = imgArray[0][0].shape[0]
    if rowsAvailable:
        for x in range(rows):
            for y in range(cols):
                img = imgArray[x][y]
                img = cv2.resize(img, (width, height))
                if len(img.shape) == 2:
                    img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
                imgArray[x][y] = img
        hor = [np.hstack(imgArray[x]) for x in range(rows)]
        return np.vstack(hor)
    else:
        imgArray = [cv2.resize(i, (width, height)) for i in imgArray]
        return np.hstack(imgArray)

In [ ]:
def getContours(img, imgContour):
    global dir, detected_area
    detected_area = 0
    contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
        area = cv2.contourArea(cnt)
        areaMin = cv2.getTrackbarPos("Area", "Parameters")
        if area > areaMin:
            detected_area = area
            x, y, w, h = cv2.boundingRect(cnt)
            cx = x + w // 2
            cy = y + h // 2
            if cx < width//2 - deadZone: dir = 1
            elif cx > width//2 + deadZone: dir = 2
            elif cy < height//2 - deadZone: dir = 3
            elif cy > height//2 + deadZone: dir = 4
            else: dir = 0

In [ ]:
while True:
    frame_read = me.get_frame_read()
    img = frame_read.frame
    img = cv2.resize(img, (width, height))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    imgContour = img.copy()
    imgHsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)

    h_min = cv2.getTrackbarPos("HUE Min", "HSV")
    h_max = cv2.getTrackbarPos("HUE Max", "HSV")
    s_min = cv2.getTrackbarPos("SAT Min", "HSV")
    s_max = cv2.getTrackbarPos("SAT Max", "HSV")
    v_min = cv2.getTrackbarPos("VALUE Min", "HSV")
    v_max = cv2.getTrackbarPos("VALUE Max", "HSV")

    lower = np.array([h_min, s_min, v_min])
    upper = np.array([h_max, s_max, v_max])
    mask = cv2.inRange(imgHsv, lower, upper)
    result = cv2.bitwise_and(img, img, mask=mask)
    mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)

    blur = cv2.GaussianBlur(result, (7, 7), 1)
    gray = cv2.cvtColor(blur, cv2.COLOR_RGB2GRAY)
    th1 = cv2.getTrackbarPos("Threshold1", "Parameters")
    th2 = cv2.getTrackbarPos("Threshold2", "Parameters")
    canny = cv2.Canny(gray, th1, th2)
    dilated = cv2.dilate(canny, np.ones((5, 5)), iterations=1)

    getContours(dilated, imgContour)

    if startCounter == 0:
        me.takeoff()
        startCounter = 1

    area_error = TARGET_AREA - detected_area
    fb_vel = int(np.clip(KP_FB * area_error, -MAX_FB_SPEED, MAX_FB_SPEED))
    fb_vel = max(min(fb_vel, -15) if fb_vel < 0 else max(fb_vel, 15), 0)

    frame_count += 1
    if frame_count % 3 == 0:
        me.for_back_velocity = fb_vel

    if dir == 1:
        me.yaw_velocity = -60
    elif dir == 2:
        me.yaw_velocity = 60
    elif dir == 3:
        me.up_down_velocity = 60
    elif dir == 4:
        me.up_down_velocity = -60
    else:
        me.yaw_velocity = 0
        me.up_down_velocity = 0

    if me.send_rc_control:
        me.send_rc_control(me.left_right_velocity, me.for_back_velocity, me.up_down_velocity, me.yaw_velocity)

    stacked = stackImages(0.9, [[img, result], [dilated, imgContour]])
    cv2.imshow("Tracking", stacked)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        me.land()
        break

cv2.destroyAllWindows()